In [1]:
import pandas as pd
from src.training.bert_pipeline import TrainingBertPipeline
from src.training.bert_truncate_pipeline import BertTruncatePipeline
import logging
import torch
import os

In [2]:
df = pd.read_csv("data/aes_dataset_5k_clean.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5363 entries, 0 to 5362
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          4859 non-null   object 
 1   reference_answer  5363 non-null   object 
 2   answer            5363 non-null   object 
 3   score             5363 non-null   float64
 4   normalized_score  5363 non-null   float64
 5   multibert_length  5363 non-null   int64  
 6   dataset           5363 non-null   object 
 7   dataset_num       5363 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 335.3+ KB


In [3]:
df['dataset'].value_counts()
# sag [10]
# sci [10]
# cunlp [10][13]
# stita [10]
# analisis_essay [6]

dataset
sag               2558
analisis_essay    2162
stita              333
cunlp              171
sci                139
Name: count, dtype: int64

In [4]:
# Check if the first file exists
df_result = None
if os.path.exists("experiments/results/results.csv"):
    df_result = pd.read_csv("experiments/results/results.csv")
    print(df_result['config_id'].iloc[-1])
else:
    print("File 'results.csv' does not exist.")

23


In [6]:
batch_sizes = [4, 8]
epochs_list = [5, 10]
learning_rates = [1e-5, 2e-5, 5e-5]
idx = (df_result['config_id'].iloc[-1] + 1) if df_result is not None and not df_result.empty else 0  # index untuk setiap kombinasi
ROOT_DIR = os.getcwd()

In [7]:
# model = [
#     ("bert_length", "bert-base-uncased"),
#     ("indobert_length", "indobenchmark/indobert-base-p1"),
#     ("albert_length", "albert-base-v1"),
#     ("indoalbert_length", "indobenchmark/indobert-lite-base-p2"),
#     ("longformer_length", "allenai/longformer-base-4096"),
#     ("multibert_length", "google-bert/bert-base-multilingual-uncased")
# ]

In [8]:
for batch_size in batch_sizes:
    for num_epochs in epochs_list:
        for lr in learning_rates:
            results = []
            results_epoch = []
            df_result1 = None
            # Check if the second file exists
            if os.path.exists("experiments/results/results_epoch.csv"):
                df_result1 = pd.read_csv("experiments/results/results_epoch.csv")
                print(max(df_result1['valid_qwk']))
            else:
                print("File 'results_epoch.csv' does not exist.")
            config = {
                "df": df,
                "model_name": "google-bert/bert-base-multilingual-uncased",
                "batch_size": batch_size,
                "learning_rate": lr,
                "epochs": num_epochs,
                "config_id": idx,
                "max_seq_len": 512,
                "col_length": "multibert_length",
                "best_valid_qwk": max(df_result1['valid_qwk']) if df_result1 is not None and not df_result1.empty else float("-inf")
            }

            logging.info(
                f"Running configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                f"max_seq_length={config['max_seq_len']}, epochs={num_epochs}, learning_rate={lr}"
            )
            
            print(
                f"\nRunning configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                f"max_seq_length={config['max_seq_len']}, epochs={num_epochs}, learning_rate={lr}"
            )
            
            try:
                pipeline = BertTruncatePipeline(config, results, results_epoch)
                pipeline.run_training()

                # Save results
                results_path = os.path.join(ROOT_DIR, "experiments/results/results.csv")
                results_epoch_path = os.path.join(ROOT_DIR, "experiments/results/results_epoch.csv")
                TrainingBertPipeline.save_csv(results, results_path)
                TrainingBertPipeline.save_csv(results_epoch, results_epoch_path)
            except Exception as e:
                logging.error(f"Error in config_id={idx}: {str(e)}")
                print(f"Error in config_id={idx}: {str(e)}")
                torch.cuda.empty_cache()
            finally:
                # Clear GPU memory after every configuration
                del pipeline.model
                del pipeline.tokenizer
                del pipeline.optimizer
                torch.cuda.empty_cache()

            idx += 1

0.9251343231432688

Running configuration: config_id=24, model_name=google-bert/bert-base-multilingual-uncased, batch_size=4, max_seq_length=512, epochs=5, learning_rate=1e-05
split dataset run...
create dataset run...
create dataloader run...
create dataloader done...
====== Training Epoch 1/5 ======


Token indices sequence length is longer than the specified maximum sequence length for this model (1001 > 512). Running this sequence through the model will result in indexing errors
c:\Users\User\Documents\Code\env\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Train Loss: 0.0482, Train QWK: 0.6669, Train Pearson: 0.7562
Validation Loss: 0.0378, Validation QWK: 0.8624, Validation Pearson: 0.8893
====== Training Epoch 2/5 ======
Train Loss: 0.0283, Train QWK: 0.7380, Train Pearson: 0.8617
Validation Loss: 0.0203, Validation QWK: 0.9093, Validation Pearson: 0.9096
====== Training Epoch 3/5 ======
Train Loss: 0.0197, Train QWK: 0.7590, Train Pearson: 0.9046
Validation Loss: 0.0212, Validation QWK: 0.9046, Validation Pearson: 0.9082
====== Training Epoch 4/5 ======
Train Loss: 0.0156, Train QWK: 0.8020, Train Pearson: 0.9248
Validation Loss: 0.0200, Validation QWK: 0.9109, Validation Pearson: 0.9120
====== Training Epoch 5/5 ======
Train Loss: 0.0106, Train QWK: 0.8348, Train Pearson: 0.9497
Validation Loss: 0.0276, Validation QWK: 0.8924, Validation Pearson: 0.9179
Test Loss: 0.0269, Test QWK: 0.8994, Test Pearson: 0.9231
0.9251343231432688

Running configuration: config_id=25, model_name=google-bert/bert-base-multilingual-uncased, batch_size=4,

Token indices sequence length is longer than the specified maximum sequence length for this model (1001 > 512). Running this sequence through the model will result in indexing errors


split dataset run...
create dataset run...
create dataloader run...
create dataloader done...
====== Training Epoch 1/5 ======
Train Loss: 0.0427, Train QWK: 0.6603, Train Pearson: 0.7815
Validation Loss: 0.0577, Validation QWK: 0.7814, Validation Pearson: 0.8890
====== Training Epoch 2/5 ======
Train Loss: 0.0277, Train QWK: 0.7368, Train Pearson: 0.8642
Validation Loss: 0.0251, Validation QWK: 0.8912, Validation Pearson: 0.9048
====== Training Epoch 3/5 ======
Train Loss: 0.0215, Train QWK: 0.7619, Train Pearson: 0.8963
Validation Loss: 0.0202, Validation QWK: 0.9041, Validation Pearson: 0.9066
====== Training Epoch 4/5 ======
Train Loss: 0.0167, Train QWK: 0.8072, Train Pearson: 0.9201
Validation Loss: 0.0244, Validation QWK: 0.8839, Validation Pearson: 0.9136
====== Training Epoch 5/5 ======
Train Loss: 0.0144, Train QWK: 0.8124, Train Pearson: 0.9313
Validation Loss: 0.0299, Validation QWK: 0.8767, Validation Pearson: 0.9040
Test Loss: 0.0276, Test QWK: 0.8909, Test Pearson: 0.918

...

Train Loss: 0.0456, Train QWK: 0.6439, Train Pearson: 0.7648
Validation Loss: 0.0315, Validation QWK: 0.8572, Validation Pearson: 0.8803
====== Training Epoch 2/10 ======
Train Loss: 0.0276, Train QWK: 0.7421, Train Pearson: 0.8641
Validation Loss: 0.0216, Validation QWK: 0.9026, Validation Pearson: 0.9062
====== Training Epoch 3/10 ======
Train Loss: 0.0180, Train QWK: 0.7799, Train Pearson: 0.9124
Validation Loss: 0.0201, Validation QWK: 0.9042, Validation Pearson: 0.9070
====== Training Epoch 4/10 ======
Train Loss: 0.0129, Train QWK: 0.8143, Train Pearson: 0.9382
Validation Loss: 0.0193, Validation QWK: 0.9092, Validation Pearson: 0.9121
====== Training Epoch 5/10 ======
Train Loss: 0.0097, Train QWK: 0.8375, Train Pearson: 0.9537
Validation Loss: 0.0267, Validation QWK: 0.8937, Validation Pearson: 0.9153
====== Training Epoch 6/10 ======
Train Loss: 0.0075, Train QWK: 0.8600, Train Pearson: 0.9643
Validation Loss: 0.0191, Validation QWK: 0.9154, Validation Pearson: 0.9181
====== T

Token indices sequence length is longer than the specified maximum sequence length for this model (1001 > 512). Running this sequence through the model will result in indexing errors


split dataset run...
create dataset run...
create dataloader run...
create dataloader done...
====== Training Epoch 1/10 ======
Train Loss: 0.0408, Train QWK: 0.6760, Train Pearson: 0.7906
Validation Loss: 0.0432, Validation QWK: 0.8357, Validation Pearson: 0.8974
====== Training Epoch 2/10 ======
Train Loss: 0.0237, Train QWK: 0.7588, Train Pearson: 0.8841
Validation Loss: 0.0245, Validation QWK: 0.8969, Validation Pearson: 0.9058
====== Training Epoch 3/10 ======
Train Loss: 0.0184, Train QWK: 0.7763, Train Pearson: 0.9115
Validation Loss: 0.0200, Validation QWK: 0.9030, Validation Pearson: 0.9075
====== Training Epoch 4/10 ======
Train Loss: 0.0154, Train QWK: 0.8046, Train Pearson: 0.9263
Validation Loss: 0.0239, Validation QWK: 0.8915, Validation Pearson: 0.9041
====== Training Epoch 5/10 ======
Train Loss: 0.0125, Train QWK: 0.8315, Train Pearson: 0.9410
Validation Loss: 0.0259, Validation QWK: 0.8934, Validation Pearson: 0.9091
====== Training Epoch 6/10 ======
Train Loss: 0.009

Token indices sequence length is longer than the specified maximum sequence length for this model (1001 > 512). Running this sequence through the model will result in indexing errors


create dataset run...
create dataloader run...
create dataloader done...
====== Training Epoch 1/10 ======
Train Loss: 0.0457, Train QWK: 0.6618, Train Pearson: 0.7673
Validation Loss: 0.0299, Validation QWK: 0.8609, Validation Pearson: 0.8843
====== Training Epoch 2/10 ======
Train Loss: 0.0262, Train QWK: 0.7588, Train Pearson: 0.8709
Validation Loss: 0.0251, Validation QWK: 0.8947, Validation Pearson: 0.9005
====== Training Epoch 3/10 ======
Train Loss: 0.0214, Train QWK: 0.7668, Train Pearson: 0.8962
Validation Loss: 0.0256, Validation QWK: 0.8911, Validation Pearson: 0.8894
====== Training Epoch 4/10 ======
Train Loss: 0.0195, Train QWK: 0.7900, Train Pearson: 0.9057
Validation Loss: 0.0214, Validation QWK: 0.8993, Validation Pearson: 0.9011
====== Training Epoch 5/10 ======
Train Loss: 0.0142, Train QWK: 0.8124, Train Pearson: 0.9319
Validation Loss: 0.0298, Validation QWK: 0.8823, Validation Pearson: 0.9044
====== Training Epoch 6/10 ======
Train Loss: 0.0121, Train QWK: 0.8295,